___
# Titanic: Tomando decisão com Teorema de Bayes

## Atividade - Classificador Naive-Bayes
___

## Índice

- [Titanic](#titanic)
- [Objetivo](#obj)
- [Classificador sem considerar qualquer característica do passageiro](#apriori)
- [Classificador Naive-Bayes com a variável `Sex`](#naivebayes)
- [Classificando os passageiros](#class)
- [Análise de sentimento](#sentimento)
- [Para casa: Estudo Complementar](#casa)
- [Classificador Naive-Bayes com a variável `Pclass`](#classPclass)
- [Separando conjunto de dados em Treinamento (train)  e Teste (test)](#testtrain)
___

<div id="titanic"></div>

## Titanic

Entre as muitas competições disponíveis na plataforma Kaggle, encontra-se a de *Machine Learning* do Titanic conforme apresentado neste [link](https://www.kaggle.com/c/titanic/data).

A base de dados apresentada no arquivo `titanic.xlsx` contém informação de 891 passageiros para os quais foram mensurados as seguintes informações (as categorias serão mantidas em inglês assim como os nomes das variáveis):
 
 * `Survived`: indica se passageiro sobreviveu ou não.
    - 0 = No,
    - 1 = Yes.
 * `Pclass`: indica a classe da passagem (*ticket*).
    - 1 = 1st, 
    - 2 = 2nd, 
    - 3 = 3rd.
 * `Sex`: sexo do passageiro. 
    - female,
    - male.
 * `Age`: idade em anos.
 * `Sibsp`: número de irmãos e/ou cônjuges a bordo do Titanic.
 * `Parch`: número de pais e/ou filhos a bordo do Titanic.
 * `Ticket`: número do *ticket*.
 * `Fare`: tarifa do passageiro.
 * `Cabin`: número da cabine.
 * `Embarked`: portão de embarque.
    - C = Cherbourg, 
    - Q = Queenstown, 
    - S = Southampton.


**Notas:**
A variável `Pclass` pode ser considerada uma *proxy* da situação socioeconômico do passageiro, ou seja, 
1st = Upper, 2nd = Middle,  3rd = Lower.


___

<div id="obj"></div>

## Objetivo

Imagine que você com as suas características estivesse no Titanic no dia no naufrágio. Será que você teria sido um sobrevivente ou não??

Assim, o objetivo dessa atividade é classificar um "novo" passageiro do Titanic como Sobrevivente ou não Sobrevivente levando em considerando (ou seja, aprendendo) o perfil dos sobreviventes e não sobreviventes que de fato estavam no Titanic.

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\matfs\Desktop\Ciência dos Dados\My_CD20-1\CD20-1\aula07


Carregando os dados do arquivo `titanic.xlsx`:

In [3]:
dados = pd.read_excel('titanic.xlsx')

Para saber quantas linhas (passageiros) e colunas (variáveis) há no *DataFrame* nomeado `dados`:

In [4]:
dados.shape

(891, 12)

___
**Definindo as variáveis qualitativas como categóricas:**

O pandas não tem como saber quais variáveis são qualitativas ao carregar os dados. Então precisamos modificar as colunas explicitamente.

Já deixamos isso pronto para você nesta avaliação:

In [5]:
dados.loc[:,'Survived'] = dados['Survived'].astype('category')
dados.loc[:,'Pclass'] = dados['Pclass'].astype('category')
dados.loc[:,'Sex'] = dados['Sex'].astype('category')
dados.loc[:,'Cabin'] = dados['Cabin'].astype('category')
dados.loc[:,'Embarked'] = dados['Embarked'].astype('category')

Agora vamos renomear os rótulos das variáveis qualitativas `Survived` e `Pclass` com comando `.cat.categories`:

In [6]:
dados.Survived.cat.categories = ['No', 'Yes']
dados.Sex.cat.categories = ['female','male']
dados.Pclass.cat.categories = ['1st','2nd','3rd']


___
***DataFrame* `dados` que deve considerar nos passos a seguir:**

In [7]:
dados.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,No,3rd,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,Yes,1st,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Yes,3rd,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


<div id="apriori"></div>

___
## Classificador sem considerar qualquer característica do passageiro

Como você poderia dar um "chute" inicial para avaliar se um novo passageiro seria ou não sobrevivente no Titanic?

Podemos usar uma probabilidade conhecida como **a priori** que não leva em consideração qualquer característica do passageiro. Essa probabilidade é dada por:

$$\qquad P(sobrevivente)\qquad \mbox{e} \qquad P(não\hspace{1mm}sobrevivente).$$

Assuma $S$ como evento ser sobrevivente e $S^c$ como evento não ser sobrevivente.

Agora, calcule essas probabilidades a partir da base de dados. Lembra que você aqui pode usar o comando `value_counts()`.

In [8]:
# Tabela de frequências relativas => Probabilidades marginais
dados.Survived.value_counts(True, sort=False).round(4)

No     0.6162
Yes    0.3838
Name: Survived, dtype: float64

Vamos guardar essas probabilidades em variáveis:

In [9]:
priori = dados.Survived.value_counts(True, sort=False)
P_Sc = priori[0]
P_S = priori[1]

print(f'Probabilidade a priori de não sobreviver {P_Sc:0.5g}')
print(f'Probabilidade a priori de sobreviver {P_S:0.5g}')

Probabilidade a priori de não sobreviver 0.61616
Probabilidade a priori de sobreviver 0.38384


E aí? Se chegasse alguém agora para ser um passageiro do Titanic, esse seria classificado como sobrevivente ou não sobrevivente?

E se chegassem 1.000 passageiros?

<div id="naivebayes"></div>

___
## Classificador Naive-Bayes

O Classificador Naive-Bayes permite que seja feito uma classificação, aqui, como sobrevivente ou não sobrevivente, porém levando em considerando **alguma(s) característica(s)** do passageiro. 

Vale ressaltar a importância de utilizar características (variáveis ou *features*) do passageiro que sejam associadas (dependentes) da variável *target* `Survived` que fornece a classificação.

Vamos entender o **Teorema de Bayes** utilizando a característica do passageiro descrita na variável `Sex`. Ainda, vamos utilizar os eventos $S$ para sobrevivente, $F$ para feminino e $M$ para masculino.

A probabilidade de **ser sobrevivente** dado cada categoria da variável `Sex` pode ser escrita como:

$$P(S|F)=\frac{P(S \cap F)}{P(F)}=\frac{P(F|S)P(S)}{P(F)}\hspace{3cm}\mbox{(I)}$$

e

$$P(S|M)=\frac{P(S \cap M)}{P(M)}=\frac{P(M|S)P(S)}{P(M)}\hspace{2.5cm}\mbox{(II)}$$


Por consequência, a probabilidade de **não ser sobrevivente** dado cada categoria da variável `Sex` pode ser escrita como:

$$P(S^c|F)=1-P(S|F)$$

e

$$P(S^c|M)=1-P(S|M)$$


Agora, vamos obter todas as probabilidade envolvidas do lado direito das equações $\mbox{(I)}$ e $\mbox{(II)}$ acima descritas.

Para facilitar, vamos obter tais probabilidades nos seguintes passos:

 * Calcular as probabilides $P(S)$, $P(S^c)$, $P(F)$ e $P(M)$. Lembra que você aqui pode usar o comando `value_counts()`.
 * Construir um conjunto de dados apenas com os sobreviventes e um outro apenas com os não sobreviventes.
 * Em cada conjunto de dados, calcular a probabilidade de ser cada categoria da variável `Sex`, ou seja, 
     * Entre os sobreviventes: calcular $𝑃(F|S)$ e $𝑃(M|S)$ a partir da base de dados. Aqui pode usar o comando `value_counts()`.
     * Entre os não sobreviventes: calcular $𝑃(F|S^c)$ e $𝑃(M|S^c)$ a partir da base de dados. Aqui pode usar o comando `value_counts()`.
          
  * Com todas essas probabilidades, calcular $P(S|F)$, $P(S^c|F)$, $P(S|M)$ e $P(S^c|M)$.

Vamos lá! **Passo-a-passo**:

* Calcule as probabilides $P(S)$, $P(S^c)$, $P(F)$ e $P(M)$. Lembra que você aqui pode usar o comando `value_counts()`.

In [10]:
prob_surv = dados.Survived.value_counts(True, sort=False)
P_Sc = prob_surv[0]
P_S = prob_surv[1]

print(f'Probabilidade a priori de não sobreviver: {P_Sc:0.5g}')
print(f'Probabilidade a priori de sobreviver: {P_S:0.5g}')

Probabilidade a priori de não sobreviver: 0.61616
Probabilidade a priori de sobreviver: 0.38384


In [11]:
dados.Sex.value_counts(True, sort=False)

female    0.352413
male      0.647587
Name: Sex, dtype: float64

In [12]:
prob_sex = dados.Sex.value_counts(True, sort=False)
P_F = prob_sex[0]
P_M = prob_sex[1]

print(f'Probabilidade de ser do sexo feminino: {P_F:0.5g}')
print(f'Probabilidade de ser do sexo masculino: {P_M:0.5g}')

Probabilidade de ser do sexo feminino: 0.35241
Probabilidade de ser do sexo masculino: 0.64759


 * Construa um conjunto de dados apenas com os sobreviventes e um outro apenas com os não sobreviventes.
 

In [13]:
dados_S = dados[dados.Survived == 'Yes'].copy()
dados_S.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,Yes,1st,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Yes,3rd,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,Yes,1st,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
8,9,Yes,3rd,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,Yes,2nd,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [14]:
dados_Sc = dados[dados.Survived == 'No'].copy()
dados_Sc.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,No,3rd,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
4,5,No,3rd,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,No,3rd,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,No,1st,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,No,3rd,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


* Entre os sobreviventes: calcule $𝑃(F|S)$ e $𝑃(M|S)$ a partir da base de dados. Aqui pode usar o comando `value_counts()`.
     

In [15]:
dados_S.Sex.value_counts(True, sort=False)

female    0.681287
male      0.318713
Name: Sex, dtype: float64

In [16]:
prob_sex_dado_S = dados_S.Sex.value_counts(True, sort=False)
P_F_dado_S = prob_sex_dado_S[0]
P_M_dado_S = prob_sex_dado_S[1]

print(f'Probabilidade de ser do sexo feminino entre os sobreviventes: {P_F_dado_S:0.5g}')
print(f'Probabilidade de ser do sexo masculino entre os sobreviventes: {P_M_dado_S:0.5g}')

Probabilidade de ser do sexo feminino entre os sobreviventes: 0.68129
Probabilidade de ser do sexo masculino entre os sobreviventes: 0.31871


* Entre os não sobreviventes: calcule $𝑃(F|S^c)$ e $𝑃(M|S^c)$ a partir da base de dados. Aqui pode usar o comando `value_counts()`.
          
  

In [17]:
dados_Sc.Sex.value_counts(True, sort=False)

female    0.147541
male      0.852459
Name: Sex, dtype: float64

In [18]:
prob_sex_dado_Sc = dados_Sc.Sex.value_counts(True, sort=False)
P_F_dado_Sc = prob_sex_dado_Sc[0]
P_M_dado_Sc = prob_sex_dado_Sc[1]

print(f'Probabilidade de ser do sexo feminino entre os sobreviventes: {P_F_dado_Sc:0.5g}')
print(f'Probabilidade de ser do sexo masculino entre os sobreviventes: {P_M_dado_Sc:0.5g}')

Probabilidade de ser do sexo feminino entre os sobreviventes: 0.14754
Probabilidade de ser do sexo masculino entre os sobreviventes: 0.85246


* Com todas essas probabilidades, calcule $P(S|F)$, $P(S^c|F)$, $P(S|M)$ e $P(S^c|M)$.

In [19]:
P_S_dado_F = (P_F_dado_S * P_S)/P_F
P_Sc_dado_F = 1 - P_S_dado_F

print(f'Probabilidade de ser sobreviventdo se feminino: {P_S_dado_F:0.5g}')
print(f'Probabilidade de ser não sobrevivente se feminino: {P_Sc_dado_F:0.5g}')

Probabilidade de ser sobreviventdo se feminino: 0.74204
Probabilidade de ser não sobrevivente se feminino: 0.25796


In [20]:
P_S_dado_M = (P_M_dado_S * P_S)/P_M
P_Sc_dado_M = 1 - P_S_dado_M

print(f'Probabilidade de ser sobreviventdo se masculino: {P_S_dado_M:0.5g}')
print(f'Probabilidade de ser não sobrevivente se masculino: {P_Sc_dado_M:0.5g}')

Probabilidade de ser sobreviventdo se masculino: 0.18891
Probabilidade de ser não sobrevivente se masculino: 0.81109


<div id="class"></div>

___
## Classificando os passageiros 

Considerando seu **Classificador Naive-Bayes**, classifique cada passageiro do *DataFrame* `dados`como `No` ou `Yes` considerando apenas a variável `Sex`.

No dataframe `dados`, guarde essas classificações em uma nova variável chamada `ClassBySex`. 

In [21]:
# Classificando as mulheres
if (P_S_dado_F > P_Sc_dado_F):
    dados.loc[(dados.Sex=='female'),'ClassBySex'] = 'Yes'
else:
    dados.loc[(dados.Sex=='female'),'ClassBySex'] = 'No'
    

In [22]:
# Classificando os homens
if (P_S_dado_M > P_Sc_dado_M):
    dados.loc[(dados.Sex=='male'),'ClassBySex'] = 'Yes'
else:
    dados.loc[(dados.Sex=='male'),'ClassBySex'] = 'No'

In [23]:
dados.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ClassBySex
0,1,No,3rd,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,No
1,2,Yes,1st,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Yes
2,3,Yes,3rd,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Yes


<div id="sentimento"></div>

___
## Análise de sentimento

Faça uma análise de sentimento cruzando a variável `ClassBySex` com a real classificação `Survived` utilizando a função `pd.crosstab`.

In [28]:
# Coloque seu código aqui
(pd.crosstab(dados.ClassBySex, dados.Survived, normalize="index", margins=True)).round(2)

Survived,No,Yes
ClassBySex,,
No,0.81,0.19
Yes,0.26,0.74
All,0.62,0.38


<div id="casa"></div>

## Para casa: Estudo Complementar

Vários pontos importantes podemos discutir com o classificador feito acima. Alguns seriam:

   * Poderia utilizar outra característica do passageiro para gerar a classificação acima como: `Pclass`, `Age`, entre outras? E ainda elas poderiam estar todas combinadas?
   * Foi avaliada a qualidade de classificação (Análise de Sentimento) considerando a mesma base de dados que criou o classificador. Como saber se um novo passageiro será classificado com a mesma qualidade medida na Análise de Sentimento anterior?

ESCREVA SUA RESPOSTA AQUI

___

Ainda, faça as tarefas a seguir:


<div id="classPclass"></div>

### Classificador Naive-Bayes com a variável `Pclass`

Refaça todas as etapas acima feita com a variável `Sex`, mas agora considerando a variável `Pclass`. Essa outra característica do passageiro classifica melhor se passageiro será ou não sobrevivente?


In [29]:
# Coloque seu código aqui

<div id="testtrain"></div>

### Separando conjunto de dados em Treinamento (train)  e Teste (test)

Para aplicar a teoria de Machine Learning (ML), é necessário separar a base de dados em duas partes: treinamento e teste.

 * **Treinamento**: usualmente contém 75% das linhas (aqui, considere 670 passageiros) para treinar o modelo. Ou seja, qual perfil do passageiro tende a ser sobrevivente e qual perfil, não sobreviveu.
 * **Teste**: as demais linhas (aqui, 221 passageiros). Esses passageiros deverão ser classificacos como sobrevivente ou não sobrevivente segundo modelo treinado. Essa classificação via modelo deve ser confrontada com real classificação fazendo uma Análise de Sentimento, como é chamado pela área de ML.
 
Faça a separação de 670 passageiros para ser da base treinamento e os demais, como teste. Faça essa separação de forma aleatória.

O comando `shuffle` da biblioteca `sklearn.utils` pode ser uma alternativa, por ora, para realizar essa separação.

In [30]:
from sklearn.utils import shuffle

In [31]:
dados2 = dados.copy()
shuffle(dados)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ClassBySex
142,143,Yes,3rd,"Hakkarainen, Mrs. Pekka Pietari (Elin Matilda ...",female,24.0,1,0,STON/O2. 3101279,15.8500,NaN,S,Yes
763,764,Yes,1st,"Carter, Mrs. William Ernest (Lucile Polk)",female,36.0,1,2,113760,120.0000,B96 B98,S,Yes
738,739,No,3rd,"Ivanoff, Mr. Kanio",male,NaN,0,0,349201,7.8958,NaN,S,No
227,228,No,3rd,"Lovell, Mr. John Hall (""Henry"")",male,20.5,0,0,A/5 21173,7.2500,NaN,S,No
609,610,Yes,1st,"Shutes, Miss. Elizabeth W",female,40.0,0,0,PC 17582,153.4625,C125,S,Yes
316,317,Yes,2nd,"Kantor, Mrs. Sinai (Miriam Sternin)",female,24.0,1,0,244367,26.0000,NaN,S,Yes
848,849,No,2nd,"Harper, Rev. John",male,28.0,0,1,248727,33.0000,NaN,S,No
396,397,No,3rd,"Olsson, Miss. Elina",female,31.0,0,0,350407,7.8542,NaN,S,Yes
236,237,No,2nd,"Hold, Mr. Stephen",male,44.0,1,0,26707,26.0000,NaN,S,No
7,8,No,3rd,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,No


In [32]:
train = dados2.iloc[0:670,]
train.shape

(670, 13)

In [33]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ClassBySex
0,1,No,3rd,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,No
1,2,Yes,1st,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Yes
2,3,Yes,3rd,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Yes
3,4,Yes,1st,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Yes
4,5,No,3rd,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,No


In [34]:
test = dados2.iloc[670:,]
test.shape

(221, 13)

In [35]:
test.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ClassBySex
670,671,Yes,2nd,"Brown, Mrs. Thomas William Solomon (Elizabeth ...",female,40.0,1,1,29750,39.0,NaN,S,Yes
671,672,No,1st,"Davidson, Mr. Thornton",male,31.0,1,0,F.C. 12750,52.0,B71,S,No
672,673,No,2nd,"Mitchell, Mr. Henry Michael",male,70.0,0,0,C.A. 24580,10.5,NaN,S,No
673,674,Yes,2nd,"Wilhelms, Mr. Charles",male,31.0,0,0,244270,13.0,NaN,S,No
674,675,No,2nd,"Watson, Mr. Ennis Hastings",male,NaN,0,0,239856,0.0,NaN,S,No


A partir daqui, treine seu classificador com o *DataFrame* `train` e avalie a qualidade do seu classificador com o *DataFrame* `test`.

In [ ]:
# Coloque seu código aqui